# Transform Bedmap2 data given as GeoTiff files to netCDF files

**Reference:**

- Fretwell, P. et al. (2013). Bedmap2: improved ice bed, surface and thickness datasets for Antarctica. The Cryosphere. doi:10.5194/tc-7-375-2013

Import libraries

In [1]:
import pooch
import xarray as xr
from pathlib import Path

Define directories where the original data will be downloaded and where the netCDF files will be saved

In [2]:
results_dir = Path(".") / "results"
data_dir = Path(".") / "data"

# Create results dir if it doesn't exists
results_dir.mkdir(exist_ok=True)

Initialize a Pooch instance with the url and registry for downloading Bedmap2 data

In [3]:
POOCH = pooch.create(
    path=data_dir,
    base_url="https://secure.antarctica.ac.uk/data/bedmap2/",
    registry={
        "bedmap2_tiff.zip": "f4bb27ce05197e9d29e4249d64a947b93aab264c3b4e6cbf49d6b339fb6c67fe",
    },
)

Define a dictionary with the name of the datasets provided by Bedmap2

In [4]:
datasets = {
    "bed": dict(name="Bedrock Height", units="meters"),
    "surface": dict(name="Ice Surface Height", units="meters"),
    "thickness": dict(name="Ice Thickness", units="meters"),
    "icemask_grounded_and_shelves": dict(
        name="Mask of Grounding Line and Floating Ice Shelves"
    ),
    "rockmask": dict(name="Mask of Rock Outcrops"),
    "lakemask_vostok": dict(name="Mask for Lake Vostok"),
    "grounded_bed_uncertainty": dict(name="Ice Bed Uncertainty", units="meters"),
    "thickness_uncertainty_5km": dict(name="Ice Thickness Uncertainty", units="meters"),
    "coverage": dict(name="Distribution of Ice Thickness Data (binary)"),
    "geoid": dict(name="Geoid Height (WGS84)", units="meters"),
}

Define function useful for getting the name of the GeoTiff file for the desired dataset

In [5]:
def get_fname(dataset, fnames):
    "Return the file name corresponding to the given dataset"
    if dataset == "geoid":
        dataset_name = "gl04c_geiod_to_WGS84.tif"
    else:
        dataset_name = "bedmap2_{}.tif".format(dataset)
    result = None
    for fname in fnames:
        if Path(fname).name == dataset_name:
            result = fname
    return result

Fetch Bedmap2 original data

In [6]:
fnames = POOCH.fetch("bedmap2_tiff.zip", processor=pooch.Unzip())

Build two datasets: `grid` and `uncertainty`.

- The `grid` is a `xarray.Dataset` containing all the datasets provided by Bedmap2 except for `thickness_uncertainty_5km`, which is defined on a different set of points.

- The `uncertainty` is a `xarray.Dataset` containing only the `thickness_uncertainty_5km` data.

In [7]:
arrays = []
for dataset in datasets:
    # Read the geotiff files using rasterio
    array = xr.open_rasterio(get_fname(dataset, fnames))
    # Remove "band" dimension and coordinate
    array = array.squeeze("band", drop=True)
    # Set a name for this dataarray
    array.name = dataset
    array.attrs["long_name"] = datasets[dataset]["name"]
    # Append units for the x and y coordinates, and for the array if needed
    array.x.attrs["units"] = "meters"
    array.y.attrs["units"] = "meters"
    if "units" in datasets[dataset]:
        array.attrs["units"] = datasets[dataset]["units"]
    
    
    if dataset != "thickness_uncertainty_5km":
        # Append array to arrays list
        arrays.append(array)
    else:
        # Convert the thickness_uncertainty_5km to its own xarray.Dataset
        uncertainty = array.to_dataset()

# Merge arrays list into a single xarray.Dataset
grid = xr.merge(arrays)
# Remove arrays list for saving some memory
del arrays
# Add metadata to both datasets
metadata = {
    "title": "Bedmap2",
    "projection": "Antarctic Polar Stereographic",
    "true_scale_latitude": -71,
    "datum": "WGS84",
    "EPSG": "3031",
    "doi": "10.5194/tc-7-375-2013",
}
grid.attrs.update(metadata)
uncertainty.attrs.update(metadata)

In [8]:
grid

<xarray.Dataset>
Dimensions:                       (x: 6667, y: 6667)
Coordinates:
  * x                             (x) float64 -3.333e+06 ... 3.333e+06
  * y                             (y) float64 -3.333e+06 ... 3.333e+06
Data variables:
    bed                           (y, x) int16 ...
    surface                       (y, x) int16 ...
    thickness                     (y, x) int16 ...
    icemask_grounded_and_shelves  (y, x) int8 ...
    rockmask                      (y, x) int8 ...
    lakemask_vostok               (y, x) float64 nan nan nan nan ... nan nan nan
    grounded_bed_uncertainty      (y, x) uint16 ...
    coverage                      (y, x) int8 ...
    geoid                         (y, x) float32 ...
Attributes:
    title:                Bedmap2
    projection:           Antarctic Polar Stereographic
    true_scale_latitude:  -71
    datum:                WGS84
    EPSG:                 3031
    doi:                  10.5194/tc-7-375-2013

In [9]:
uncertainty

<xarray.Dataset>
Dimensions:                    (x: 1361, y: 1361)
Coordinates:
  * y                          (y) float64 3.4e+06 3.395e+06 ... -3.4e+06
  * x                          (x) float64 -3.399e+06 -3.394e+06 ... 3.401e+06
Data variables:
    thickness_uncertainty_5km  (y, x) uint16 ...
Attributes:
    title:                Bedmap2
    projection:           Antarctic Polar Stereographic
    true_scale_latitude:  -71
    datum:                WGS84
    EPSG:                 3031
    doi:                  10.5194/tc-7-375-2013

Save datasets as netCDF files.

Use `zlib` encoding for reducing their size.

In [10]:
grid.to_netcdf(
    results_dir / "bedmap2.nc",
    encoding={dataset: {"zlib": True} for dataset in grid},
)
uncertainty.to_netcdf(
    results_dir / "bedmap2_ice_uncertainty.nc",
    encoding={dataset: {"zlib": True} for dataset in uncertainty},
)